# Entrega 4 SM

En primer lugar importamos todas las librerias necesarias.

In [ ]:

# Importacion.
# import librosa
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt

## Primeros pasos: Cargar el archivo de audio (sonido estéreo vs mono)



En este paso cargamos los directorios que se van a utilizar.

In [ ]:
# Directorios que usaremos.
cwd = os.getcwd()
audio_input_path = os.path.join(cwd, os.path.join('audio', '_input'))  # cambiar '_input' por 'examples'
audio_output_path = os.path.join(cwd, os.path.join('audio', '_output'))
print(f'Directorio con los audios de entrada: {audio_input_path}')
print(f'Directorio donde guardaremos los audios generados: {audio_output_path}\n')

Existen múltiples archivos de audio, como por ejemplo:

.wav:Es un tipo de archivo sin comprimir. Esto quiere decir que ocupa más pero no pierde calidad.

.mp3:Archivo de audio comprimido.Pesa menos pero pierde calidad.

In [ ]:
# Cargamos el archivo de audio.
filename = os.path.join(cwd, 'interstellar.wav')
# audio_data, sample_rate = librosa.load(filename, sr=None, mono=False)
sample_rate, audio_data = wavfile.read(filename)
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate) # .T se pasa únicamente si es audio estéreo.

## Mostrar Caracterisicas del audio

A continuación mostramos el audio en estéreo

In [ ]:
# Mostrar informacion (sonido estéreo).
print('Datos de audio (estereo):')
print(f'- Tamaño:     {audio_data.shape}')
print(f'- 1º canal:   {audio_data[:5, 0]}...')
print(f'- 2º canal:   {audio_data[:5, 1]}...')
print(f'- Resolucion: {type(audio_data[0,0])}\n')

Calculamos la media por canal para obtener un audio mono.

In [ ]:
# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  # Column-wise.
print('Nuevos datos de audio (mono):')
print(f'- Nuevo tamaño: {new_data_mono.shape}')
print(f'- Canal unico:  {new_data_mono[:5]}...')

# Mantenemos la misma resolucion que antes.
new_data_mono = new_data_mono.astype(np.int16)
print(f'- Resolucion:   {type(new_data_mono[0])}\n')

Procedemos a guardarlo

# Guardamos el archivo mono a un fichero de tipo wav.


wavfile.write(
    filename=os.path.join(cwd, 'sample1_mono.wav'),
    rate=sample_rate,
    data=new_data_mono
)

Escuchamos el audio de nuevo para comprobar cambios.

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)

Los tipos de sonido son:

Mono:Podemos apreciar que se escucha los mismo en cada salida de auricular.

Estéreo:El sonido es distinto por cada salida del auricular.

Comprobamos lo que ocupa cada archivo

In [ ]:
!ls -sh interstellar.wav
!ls -sh sample1_mono.wav

El tamaño, es practicamente la mitad.

In [ ]:
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz\n')

# Toma de muestras de audio

Muestra:Una muestra es un istante de tiempo concreto de una onda

Amplitud de onda: Podemos decir que es el alargamiento, ensanchamiento y distension que toma una onda

Fecuencia de muestreo:Supone la cantidad de muestras que tomamos por segundo. A más muestras más probable es que replique la oda original. Se mide en HZ y a mayor frecuencia, mayor cantidad de detalles.

Por tanto a mayor frecuencia de muestro,mayor calidad obtenemos, pero como consecuencia tendremos u archivo mucho más pesado.

# ¿A qué valor fijamos la frecuencia de muestreo?

La teoría de Nyquist establece que *necesitamos una frecuencia de muestreo igual al doble de la frecuencia más alta de una señal para capturar todas las frecuencias de la misma*.
El ciclo de onda singular tiene siempre un valor de amplitud positivo y otro negativo. Necesarias para obtener la longitud de onda de cada ciclo.
Como mínimoo deberemos mostrar cada ciclo dos veces.

### Aliasing

Es cuando la frecuencia de muestreo es inferior al doble de la máxima frecuencia contenida en el espectro de la señal analógica

### Profundidad de bits

Es la resolucion de captura de una señal de audio en relacion a la amplitud.

### Tasa de bits

Es la cantidad de datos consumidos para transmitir la secuencia de audio por unidad de tiempo

# Gráfica con el dominio de tiempo para audio mono y estéreo.

In [ ]:
ampl_values_48 = len(audio_data)

In [ ]:
# Construimos el array para el eje x que representa el tiempo de la grabación.
# Tiene la forma: np.arange(Vi, Vf, P). Explicado a continuación.
t1 = np.arange(0, ampl_values_48/sample_rate, 1/sample_rate)

In [ ]:
# Creamos la figura.
fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Solo mostramos las primeras 50 muestras de amplitud (por claridad).
end = 50

# Señal a 48 kHz.
ax[0].plot(t1[:end], audio_data[:end], marker='X')
ax[0].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate} Hz')
ax[0].set_ylabel('Amplitud')
ax[0].grid(True)

# Señal a 48 kHz.
ax[1].plot(t1[:end], new_data_mono[:end], marker='X')
ax[1].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate} Hz')
ax[1].set_ylabel('Amplitud')
ax[1].grid(True)

# Mostramos la figura.
plt.tight_layout()
plt.show()


En esta gráfica, utilizamos los 2 tipos de audio generados anteriormente.
La primera onda tiene más precisión debido a que tiene mas muestras. Por lo que es más cercana a la onda original.
Con ax[0] y ax[1] tenemos los instantes de tiempos de cada onda.

### Análisis de Fourier

Se trata de descomponer una señal, con esto obtenemos información de su composición.

### Dominio de la frecuencia: Transformada de Fourier (FFT)

In [ ]:
# La longitud del array de datos y el
# sample rate (frecuencia de muestreo).
n = len(new_data_mono)
Fs = sample_rate

# Working with stereo audio, there are two channels in the audio data.
# Let's retrieve each channel seperately:
# ch1 = np.array([data[i][0] for i in range(n)]) #channel 1
# ch2 = np.array([data[i][1] for i in range(n)]) #channel 2
# We can then perform a Fourier analysis on the first
# channel to see what the spectrum looks like.

# Calculando la Transformada Rapida de Fourier (FFT) en audio mono.
ch_Fourier = np.fft.fft(new_data_mono)  # ch1

# Solo miramos frecuencia por debajo de Fs/2
# (Nyquist-Shannon) --> Spectrum.
abs_ch_Fourier = np.absolute(ch_Fourier[:n//2])

# Graficamos.
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud', labelpad=10)
plt.xlabel('$f$ (Hz)', labelpad=10)
plt.show()

Con esto, eliminamos las ondas que menos aportan, con esto lo que buscamos en que ocupe menor espacio, ya que la suma de las ondas más importantes dan como resultado la onda original.
En este caso al eliminar las más residuales, perderemos un poco de calidad pero pesarán menos.

## Energia del espectrograma y frecuencia de corte

Frecuencia umbral:f0
Solo nos quedamos con las frecuencias que estén por debajo de dicho valor.
El epsilon es la energia del espectro. Es la parte la cual no conservamos.



In [ ]:
# Definimos epsilon: la parte de la energia
# del espectro que no conservamos.
eps = [1e-5, .02, .041, .063, .086, .101, .123]

# Jugamos con los valores de epsilon (CAMBIAD ESTO).
eps = eps[1]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para esta energia.
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# Integral de la frecuencia --> energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara (array booleano) que compara el valor
# de corte con la energia del espectro.
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# La frecuencia f0 por la que cortamos el espectro.
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos.
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

## Reducción de la resolución de muestreo (*downsampling*)

Se trata de dividir una determinada cantidad de información entre un entero o una fracción racional.
Lo que estamos haciendo es utilizar el audio mono y comprimirlo.
en data tendremos  la informacion de new_data_mono, que es el audio en mono.


In [ ]:
# Definimos los nombres de los audios comprimidos.
wav_compressed_file = "new_data_mono_compressed.wav"

# Calculamos el factor D de downsampling.
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

# Obtenemos los nuevos datos (slicing with stride).
new_data = new_data_mono[::D]

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join(cwd, wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

In [ ]:
# Cargamos el nuevo archivo.
new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(cwd, wav_compressed_file))

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate)

In [ ]:
!ls -sh new_data_mono_compressed.wav

## Espectrograma

El espectrograma es el resultado de calcular el espectro de una señal por ventanas de tiempo de la misma.

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(new_data_mono, NFFT=1024, Fs=sample_rate, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

Como podemos ver, el primer espectrograma llega a 20000 y en la segunda a 5000, reduce la calidad, pero el tamaño es mucho menor.
Al estar el audio comprimido, el espacio es menor que el original.La frecuencia se ve modificada.

El tamaño de ambos archivos es el siguiente:

In [ ]:
!ls -sh new_data_mono_compressed.wav
!ls -sh sample1_mono.wav

A continuación mostramos los dos audios:

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate)

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)